In [1]:
import os
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/olivieroandreussi/PWSCF/qe-7.2/bin/pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from ase.build import molecule
from ase.visualize import view
atoms = molecule('H2O')
view(atoms, viewer="x3d")

In [4]:
from ase.calculators.espresso import Espresso

pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

input_data = {
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
    },
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos'
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta':0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [5]:
energy = atoms.get_potential_energy()
print(f"E = {energy:.3f}")

--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 3 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------
[geco.boisestate.edu:92128] 3 more processes have sent help message help-mpi-api.txt / mpi-abort
[geco.boisestate.edu:92128] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


CalculationFailed: Calculator "espresso" failed with command "mpirun -np 4 /Users/olivieroandreussi/PWSCF/qe-7.2/bin/pw.x -in espresso.pwi > espresso.pwo" failed in /Users/olivieroandreussi/Dropbox/Workshops/i-COMSE_2023/Tutorials/Environ-Tutorials/Materials/Environ_Basics with error code 1

The following cell uses bash to generate a new file in the working folder. This file, named environ.in, is the input used by Environ to setup the environment conditions.

In [3]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 0.1
   environ_type = 'vacuum'
   env_electrostatic = .true.
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-9
   !
/
EOF